# Machine Learning with Flowers

In [26]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [27]:
#Prepping Data

data =[]
for dir,x,files in os.walk("flower_photos"):
    label= dir.split('/')[-1]
    
    for file in files:
        path = os.path.join(dir,file)

        data.append([path,label])

df=pd.DataFrame(data,columns=["file_path","label"])
print(df)

                                             file_path   label
0       flower_photos/roses/16209331331_343c899d38.jpg   roses
1        flower_photos/roses/5777669976_a205f61e5b.jpg   roses
2      flower_photos/roses/4860145119_b1c3cbaa4e_n.jpg   roses
3       flower_photos/roses/15011625580_7974c44bce.jpg   roses
4     flower_photos/roses/17953368844_be3d18cf30_m.jpg   roses
...                                                ...     ...
3665     flower_photos/tulips/134143359_71fa8dd9a4.jpg  tulips
3666    flower_photos/tulips/3637371174_a8dfcc1b35.jpg  tulips
3667  flower_photos/tulips/6948239566_0ac0a124ee_n.jpg  tulips
3668    flower_photos/tulips/2834890466_1cf220fba1.jpg  tulips
3669   flower_photos/tulips/13953090784_0c7d7a904e.jpg  tulips

[3670 rows x 2 columns]


In [15]:
datagen = ImageDataGenerator(
    validation_split=0.2
)

In [16]:
train_gen=datagen.flow_from_dataframe(
    dataframe = df,
    x_col='file_path',
    y_col='label',
    target_size=(224,224),
    color_mode= 'rgb',
    batch_size =32,
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training'
)

valid_gen=datagen.flow_from_dataframe(
    dataframe = df,
    x_col='file_path',
    y_col='label',
    target_size=(224,224),
    color_mode= 'rgb',
    batch_size =32,
    class_mode = 'categorical',
    shuffle = True,
    subset = 'validation'
)

Found 2936 validated image filenames belonging to 5 classes.
Found 734 validated image filenames belonging to 5 classes.


In [19]:
base_model = EfficientNetB3(
    include_top= False, 
    weights='imagenet',
    input_shape=(224, 224, 3), 
    pooling='max'
)

model = tf.keras.models.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(rate=.45, seed=123),
    layers.Dense(5, activation='softmax')
])


model.compile(
    Adamax(learning_rate= .0001), 
    loss = 'categorical_crossentropy',
    metrics=[
        'acc', 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall(), 
        tf.keras.metrics.AUC()
    ]
)

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [28]:
early_stopping= EarlyStopping(
    patience=10,
    min_delta=0,
    monitor='val_loss',
    verbose =0,
    restore_best_weights=True,
    baseline =None

)

In [30]:
model.fit(
    train_gen,
    validation_data =valid_gen,
    epochs = 5,
    validation_steps =None,
    shuffle = False,
    callbacks=early_stopping,
    verbose =1,
)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 326s 4s/step - acc: 0.7266 - auc: 0.9286 - loss: 0.7613 - precision: 0.7891 - recall: 0.6628 - val_acc: 0.1172 - val_auc: 0.4335 - val_loss: 3.6713 - val_precision: 0.1246 - val_recall: 0.1076
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 322s 3s/step - acc: 0.8420 - auc: 0.9738 - loss: 0.4513 - precision: 0.8806 - recall: 0.8027 - val_acc: 0.1199 - val_auc: 0.3965 - val_loss: 4.6180 - val_precision: 0.1193 - val_recall: 0.1104
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 324s 4s/step - acc: 0.8878 - auc: 0.9870 - loss: 0.3152 - precision: 0.9145 - recall: 0.8609 - val_acc: 0.1172 - val_auc: 0.3814 - val_loss: 5.0574 - val_precision: 0.1194 - val_recall: 0.1131
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 315s 3s/step - acc: 0.9198 - auc: 0.9938 - loss: 0.2251 - precision: 0.9418 - recall: 0.8988 - val_acc: 0.1172 - val_auc: 0.3753 - val_loss: 5.4247 - val_precision: 0.1223 - val_recall: 0.1158
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 291s 3s/step - acc: 0.9369 - auc: 0.9949 - 

In [32]:
model.save('flowers.keras')

In [37]:
import cv2
from tensorflow.keras.saving import load_model
import numpy as np




model = load_model("flowers.keras")

input_image = cv2.imread('flower_photos/roses/174109630_3c544b8a2f.jpg')

input_image_resize = cv2.resize(input_image,(224,224))

input_image_scaled =input_image_resize/225
image_reshaped =np.reshape(input_image_scaled,[1,224,224,3])


print(train_gen.class_indices.items())
input_prediction = model.predict(image_reshaped)

print(input_prediction)
print("Your Flower is a a ",np.argmax(input_prediction))

dict_items([('daisy', 0), ('dandelion', 1), ('roses', 2), ('sunflowers', 3), ('tulips', 4)])
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.79082114 0.18642455 0.01476624 0.00478837 0.00319975]]
Your Flower is a a  0
